# Using GPano.py

In [12]:
import sys  
sys.path.insert(0, r'/media/huan/HD4T/Research/StreetView/Google_street_view')
sys.path.insert(0, r'K:\Research\StreetView\Google_street_view')

import GPano
# import Fill_sidewalks as sw
import pandas as pd
import os
import glob

gpano = GPano.GPano()
gsv = GPano.GSV_depthmap()


def list_to_file(a_list, file_name):
    with open(file_name, mode='w', encoding='utf-8') as myfile:
        myfile.write('\n'.join(a_list))
        myfile.write('\n')

FileNotFoundError: [Errno 2] No such file or directory: 'Google_map_key.ini'

In [13]:
panoId_map = r'/media/huan/SSD/Datasets/ocean_city/yolo/inference/rename.csv'
depth_map_dir = r'/media/huan/SSD/Datasets/ocean_city/yolo/inference/depthmap'

detected_dir = r'/media/huan/SSD/Datasets/ocean_city/yolo/inference/output'



if not os.path.exists(depth_map_dir):
    os.mkdir(depth_map_dir)

FileNotFoundError: [WinError 3] The system cannot find the path specified: '/media/huan/SSD/Datasets/ocean_city/yolo/inference/depthmap'

In [3]:
panoId_file = r'/media/huan/SSD/Datasets/ocean_city/yolo/inference/rename.csv'

In [ ]:
panoId_df = pd.read_csv(panoId_file)

In [ ]:
panoId_dict = {}
for idx, row in panoId_df.iterrows():
    jpg = row['jpg']
    panoId = row['panoid']
    jpg_pos = panoId.find(".jpg")
    panoId_dict[jpg] = panoId[:jpg_pos]
 

In [ ]:
jpgs = glob.glob(os.path.join(detected_dir, "*.jpg"))
distances = []
for jpg in jpgs:
    basename = os.path.basename(jpg)
    panoId = panoId_dict[basename]
#     jdata = gpano.getJsonfrmPanoID(panoId, dm=1, saved_path=depth_map_dir)
#     dm = gsv.getDepthmapfrmJson(jdata)
    
    img = gsv.saveDepthMap_frm_panoId(panoId, saved_path=depth_map_dir)
    shape = img.size
#     print(img)
    lowest_row = max(0, shape[1] - 1)
    central_col = int(shape[0]/2)
    print(lowest_row, central_col)
    d = img.load()[central_col, lowest_row]
    line = ",".join([panoId, str(d)])
    distances.append(line)

In [ ]:
list_to_file(jpgs, os.path.join(r"/media/huan/SSD/Datasets/ocean_city/yolo/inference/jpgs.txt"))
list_to_file([str(d) for d in distances], os.path.join(r"/media/huan/SSD/Datasets/ocean_city/yolo/inference/distances.txt"))

In [2]:
%cd /media/huan/HD4T/Research/StreetView/Google_street_view

/media/huan/HD4T/Research/StreetView/Google_street_view


In [3]:
!python Shoot_objects.py

Traceback (most recent call last):
  File "Shoot_objects.py", line 1, in <module>
    import GPano
  File "/media/huan/HD4T/Research/StreetView/Google_street_view/GPano.py", line 23, in <module>
    from geopy.distance import geodesic
ModuleNotFoundError: No module named 'geopy'


# Calculate the height


## download depthmap

In [6]:
depth_map_dir = r'/media/huan/SSD/Datasets/HamptonRoads/depthmaps'

detected_dir = r'/media/huan/HD4T/Dataset/HamptonRoads/Google_thumbnails_detected'



if not os.path.exists(depth_map_dir):
    os.mkdir(depth_map_dir)

In [7]:
def extract_panoId_frm_name(file_name):
    fileds = file_name.split("_")
    prefix = fileds[0]
     
    theta_phi = fileds[-2] + "_" + fileds[-1]
    
    panoId = file_name
    
    panoId = panoId.replace(theta_phi, "")
    start = 0
    if prefix.isnumeric():
#         print(prefix)
        panoId = panoId.replace(prefix, "")
        start = 1
     
    
    panoId = panoId[start:-1]
    
    return panoId
    

def extract_panoId_frm_Latlon_name(file_name):
    fileds = file_name.split("_")
    prefix = fileds[0]

    lat = fileds[1]
    lon = fileds[2]
    

    panoId, panoLon, panoLat = gpano.getPanoIDfrmLonlat(lon, lat)
 

    return panoId

# extract_panoId_frm_name("9dio_Mcjwvt65YIJDaXn0Q_0_90")    
extract_panoId_frm_Latlon_name("3780_36.929239_-76.201919_0_288.92")    

'Mk_eB3E-pI5_Bz37T5t3UA'

2731

In [10]:
from tqdm import tqdm

jpgs = glob.glob(os.path.join(detected_dir, "*.jpg"))


distances = []
for jpg in tqdm(jpgs):
    try:
        basename = os.path.basename(jpg)

        panoId = extract_panoId_frm_Latlon_name(basename.replace(".jpg", ""))
          
#         print(panoId)

        dm_name = os.path.join(depth_map_dir, panoId + ".tif")
#         print(dm_name)
        if os.path.exists(dm_name):
            continue

        img = gsv.saveDepthMap_frm_panoId(panoId, saved_path=depth_map_dir)
        shape = img.size
        print(panoId)
        lowest_row = max(0, shape[1] - 1)
        central_col = int(shape[0]/2)
#         print(lowest_row, central_col)
        d = img.load()[central_col, lowest_row]
        line = ",".join([panoId, str(d)])
        distances.append(line)
    except Exception as e:
        print("Error: ", jpg, e)
        


  1%|          | 28/2731 [00:06<10:16,  4.39it/s]


KeyboardInterrupt: 

## camera height list

In [78]:
camera_h_file = os.path.join(depth_map_dir, "camera_heights.csv")
distances.insert(0, "panoId,camera_height")
list_to_file(distances, camera_h_file)


'/media/huan/SSD/Datasets/HamptonRoads/depthmaps/camera_heights.csv'

In [80]:
camera_df = pd.read_csv(camera_h_file)

camera_h_dict = {}
for idx, row in camera_df.iterrows():
    file_name = str(row['panoId'])
    height =  row['camera_height']
    camera_h_dict[file_name] = height

In [82]:
# camera_h_dict

## Get DEM dict

In [107]:
camera_dem_file = r'/media/huan/SSD/Datasets/HamptonRoads/camera_dem.csv'

DEM_df = pd.read_csv(camera_dem_file)

camera_DEM_dict = {}
for idx, row in DEM_df.iterrows():
#     print(row)
    panoId = str(row['panoId'])
    dem =  row['DEM_ft']
    camera_DEM_dict[panoId] = dem * 0.3048

In [108]:
camera_DEM_dict

{'--4g0Jz__5drFDnBr3D13w': 1.7276063999999998,
 '-50sPDLEhQhPD_dmCZ8yUg': 2.6404824000000002,
 '-68BCYBHNffBOAnNc2J96g': 1.6901160000000002,
 '-9kAcnTXKB_5whqGmzYjxg': 0.7763256,
 '-9WW0orMVfRhX70X-QRphA': 2.6471880000000003,
 '-B5pIi-Kjr14Q3IpnNyjZA': 0.9354312,
 '-bcgPwKJ6SHooJUi0h62Lg': 1.4474951999999999,
 '-byDrR52ijoqKGrCX7MblA': 2.3222712,
 '-CD3eAzKQqz8BCBiUecD3A': 11.648846400000002,
 '-csuHcyAnmgXIxHvaKPL0g': 2.1765768000000003,
 '-d0DvFqBHje8tCG1VNE2ng': 2.1204936000000005,
 'HBK9qCRoQLrDSHgw0A6xDQ': 1.054608,
 'Hb_Nn5ws6ppEO6SLjynhNg': 2.6734008000000005,
 'Hc72gwrxv9S_NrPnPzFfEA': 1.7242536000000002,
 'hCLb2_vr1FdvQMAkKREH8Q': 1.7748504,
 'HCMapyDNjj0ERcXYV0OLdw': 1.7263872,
 'HCwAgitcV9ntsExHgpjXWQ': 2.0519136000000002,
 'Hd7-4tNbTc0FHEA0AxdJdQ': 1.7004792,
 'HdKirGsT2tR5H8ZQNWkCuA': 2.1079968,
 'HdyIPxCeb1vy52U-hJYi5Q': 1.8388584,
 'He4-uK3mnwjwxa7OAjNs2Q': 1.2691872,
 'hFMO9LGVYYMPJFiIR_Jc6w': 0.877824,
 'RhQnF8utgo3v7U40DqPMPg': 1.9092672,
 'rI4tcP5-b9AQku0O4kkHsQ': 1.

## create relationship between Ground-truth and panorama

In [130]:
ground_truth = r'/media/huan/SSD/Datasets/HamptonRoads/EC-lat-lon.csv'
EC_df = pd.read_csv(ground_truth)
EC_df["panoId"] = ''
EC_df

,ID,LAT,LON,panoId
0,1,36.733735,-76.295499,0
1,2,36.730780,-76.235625,9dio_Mcjwvt65YIJDaXn0Q
2,3,36.730565,-76.235618,e-IAe215jteHyBVMV60wYw
3,4,36.775462,-76.323840,dseu1kHtzrPCmHqmdx8gWA
4,5,36.775673,-76.323941,Ad-v-htV8SoT9M8Ykh7hLg
...,...,...,...,...
3799,3800,36.915429,-76.824209,0
3800,3801,36.916143,-76.830666,0
3801,3802,36.975370,-76.859817,Q_GuzcSSbsMTqtaJlEFIlQ
3802,3803,36.975384,-76.858795,-CD3eAzKQqz8BCBiUecD3A


In [ ]:
jsons_file = r'/media/huan/SSD/Datasets/HamptonRoads/json2500_2_DEM_camera_h.csv'
json_df = pd.read_csv(ground_truth)
EC_df["panoId"] = ''
EC_df

In [123]:
# lines = []
# lines.append("EC_id,LAT,LON")
EC_df["panoId"] = ''
for idx, row in EC_df.iterrows():
    
  
    lat = row["LAT"]
    lon = row["LON"]
    

     
    panoId, panoLon, panoLat = gpano.getPanoIDfrmLonlat(lon, lat)
    EC_df.at[idx,"panoId"] = panoId
    
    if idx % 50 == 0:
        print("Processing:", idx, EC_df.iloc[idx]["panoId"])
        
    
EC_df

Processing: 0 0
Processing: 50 X-01bW8PT3h0srAqVQwrnA
Processing: 100 u9ZJRPA_1kXYBToOCxW0cA
Processing: 150 0
Processing: 200 0
Processing: 250 0
Processing: 300 7KlOToR4ptvH5QIIHCls7A
Processing: 350 HmGCzggfW7I0b4-_gH7Gxg
Processing: 400 8Uj7IPc-4dbmyBSeY-LO6A
Processing: 450 TKcXt9wMCHEsaqtf-GHrFA
Processing: 500 0
Processing: 550 opcQEenMSqaVo_ijsLiQiA
Processing: 600 0
Processing: 650 0
Processing: 700 0
Processing: 750 ehmUMZYbuyvCpO10qgWrow
Processing: 800 0
Processing: 850 0
Processing: 900 0
Processing: 950 dkalMMKZu9vI9MuGrXAJog
Processing: 1000 1nObxvt2Wnf4FUtMZ8Oaow
Processing: 1050 0
Processing: 1100 0
Processing: 1150 4kOC6qr-MtL-qKk1EFGK7A
Processing: 1200 0
Processing: 1250 0
Processing: 1300 0
Processing: 1350 0
Processing: 1400 0
Processing: 1450 0
Processing: 1500 0
Processing: 1550 J-KA18FMbI3p8K5r19A6Tg
Processing: 1600 0
Processing: 1650 NzW61QUDF8w1vRS27wVYyQ
Processing: 1700 0
Processing: 1750 A5WMGWIqwa0NJZ2OlaF4Ag
Processing: 1800 C98rwMPwk8aJYDkQfytpAw
Proce

,ID,LAT,LON,panoId
0,1,36.733735,-76.295499,0
1,2,36.730780,-76.235625,9dio_Mcjwvt65YIJDaXn0Q
2,3,36.730565,-76.235618,e-IAe215jteHyBVMV60wYw
3,4,36.775462,-76.323840,dseu1kHtzrPCmHqmdx8gWA
4,5,36.775673,-76.323941,Ad-v-htV8SoT9M8Ykh7hLg
...,...,...,...,...
3799,3800,36.915429,-76.824209,0
3800,3801,36.916143,-76.830666,0
3801,3802,36.975370,-76.859817,Q_GuzcSSbsMTqtaJlEFIlQ
3802,3803,36.975384,-76.858795,-CD3eAzKQqz8BCBiUecD3A


In [124]:
EC_df.to_csv(r"/media/huan/SSD/Datasets/HamptonRoads/EC-latlon-panoId.csv", index=False)

In [ ]:
from PIL import Image

distances = []
for dm in tqdm(dms):
    try:
        img = Image.open(dm)
        shape = img.size
        
        lowest_row = max(0, shape[1] - 1)
        central_col = int(shape[0]/2)
#         print(lowest_row, central_col)
        d = img.load()[central_col, lowest_row]
        panoId = os.path.basename(dm).replace(".tif", '')
        
        line = ",".join([panoId, str(d)])
        distances.append(line)
        print(line)
    except Exception as e:
        print(e, dm)
        
list_to_file(distances, camera_h_file)

## re-construct the camera-height file

In [61]:
dms = glob.glob(depth_map_dir + '/*.tif')#
# dms

In [70]:
from PIL import Image

distances = []
for dm in tqdm(dms):
    try:
        img = Image.open(dm)
        shape = img.size
        
        lowest_row = max(0, shape[1] - 1)
        central_col = int(shape[0]/2)
#         print(lowest_row, central_col)
        d = img.load()[central_col, lowest_row]
        panoId = os.path.basename(dm).replace(".tif", '')
        
        line = ",".join([panoId, str(d)])
        distances.append(line)
        print(line)
    except Exception as e:
        print(e, dm)
        
list_to_file(distances, camera_h_file)

  5%|▍         | 114/2496 [00:00<00:04, 574.27it/s]

--4g0Jz__5drFDnBr3D13w,2.9326531887054443
-50sPDLEhQhPD_dmCZ8yUg,2.408653736114502
-68BCYBHNffBOAnNc2J96g,2.9158411026000977
-9kAcnTXKB_5whqGmzYjxg,2.9188592433929443
-9WW0orMVfRhX70X-QRphA,2.919037103652954
-B5pIi-Kjr14Q3IpnNyjZA,2.8027329444885254
-bcgPwKJ6SHooJUi0h62Lg,2.409454107284546
-byDrR52ijoqKGrCX7MblA,2.409414529800415
-CD3eAzKQqz8BCBiUecD3A,2.9181323051452637
-csuHcyAnmgXIxHvaKPL0g,2.8363006114959717
-d0DvFqBHje8tCG1VNE2ng,2.919766426086426
HALENuOEnhegwIXMZd7G0g,2.9274137020111084
HBK9qCRoQLrDSHgw0A6xDQ,2.4737210273742676
Hb_Nn5ws6ppEO6SLjynhNg,2.9174997806549072
Hc72gwrxv9S_NrPnPzFfEA,2.901496648788452
hCLb2_vr1FdvQMAkKREH8Q,2.828583002090454
HCMapyDNjj0ERcXYV0OLdw,2.916200876235962
HCwAgitcV9ntsExHgpjXWQ,2.91363525390625
Hd7-4tNbTc0FHEA0AxdJdQ,2.385998010635376
HdKirGsT2tR5H8ZQNWkCuA,2.911926746368408
HdyIPxCeb1vy52U-hJYi5Q,2.918844699859619
He4-uK3mnwjwxa7OAjNs2Q,2.919020414352417
RhQnF8utgo3v7U40DqPMPg,2.8537349700927734
rI4tcP5-b9AQku0O4kkHsQ,2.9402225017547607
rIKobd

  9%|▊         | 214/2496 [00:00<00:04, 533.18it/s]

UB9OA56c2yp7ST5B5AhYuA,2.408932685852051
uBDq0NVhCtz0gADI9Tgiog,2.9166626930236816
ubjQJI5u1pxXMB6jJTFyyg,2.8891396522521973
uCAHS014RE2Gui-IZqIRtg,2.919067621231079
UcQSaro_AlhSphvGE6gmFg,2.9770147800445557
zFFFVOWukv6nqkR69Uu_LQ,2.918666362762451
ZFFoAxqY00p35tRx3du8jg,3.035393714904785
ZftxARROSC6soWk_sZCs1A,3.015944719314575
ZF_S2qD0pGxi45YZQXyVEA,2.9369583129882812
ZgLSBCEg1Spqr0NxYqdyEg,2.446183204650879
zGsdk2fHexjnk9t_sOOizg,2.901470184326172
zgVm_mOovDKu-l04YqGdIg,2.408764600753784
Ezj3C0WYZtWzFTKtJzP3og,2.410039186477661
eZuhOLHZgwFryWdXJWVU1g,2.410733699798584
E_jflIfKuedLeW01PIQ0DA,2.410611391067505
f-m3DgMTfSPoWiC2Ko4CKA,2.4958314895629883
f18ZWk_7SxF2E7uy3vDKdQ,2.3662233352661133
f1tqsbk8b-7fh3fYlEl5Ng,2.9304633140563965
f1X1n5t14YVJGO6MnqLt8g,2.9339728355407715
f2m6mj-yCukmhoyQf3oRWA,2.9109435081481934
2RdofIKfSO2jRjcFwPZKhw,2.3802969455718994
2repr1LXkS44qQGWVtH-bw,2.4088449478149414
2rHC8mvE__6xLF7mUZnARA,3.0589640140533447
2Sarc4yx-PuWbUx5sJ2j4A,2.408698081970215
2UYh

 13%|█▎        | 322/2496 [00:00<00:04, 534.95it/s]

...

Ib0xVJ7pi5lcnI8g9vcg1Q,2.92829966545105
Ib68A67qsDSkXlX1FeN8gA,2.4747507572174072
ic-043VUACPInXaqbFKZzw,2.9252243041992188
ICaDKnzX46F9xEhBGNDcMA,2.9090776443481445
ICfujRwtuh7oMB6rImceew,2.409409761428833
IeKz84MQrfT5Fqnmf1ff2Q,2.8972764015197754
ToMt9ZPhcM7UqBgxFE3VsA,2.7879555225372314
toPoBK1aMKJaRQTwNvXp3Q,2.9136834144592285
tOPOxQO4uOig5bl7UD4v6g,2.99538516998291
tOsy239eEMFSWHgN2xXM0w,2.450856924057007
tOt8pKJcatGQ-dSRCmJ9LQ,2.4095840454101562
tOTHdFlkMfz71P34oHdlHA,2.4089181423187256
TpcPIodOa6v0-ates-rfzw,2.9189984798431396
tPKf1_A8x4nrYFrBcjW7GQ,2.9094297885894775
j8eC6fp2DB4D3a6nQQ85yA,2.48038387298584
J8gzrD3KLxFD3tsq8QLjSQ,2.4083099365234375
J8Q4qjFM4f-2FfkI4ui4wA,2.9173285961151123
J9ldT7wZ9poIoSdEQvOq4g,2.9157516956329346
j9TnZk_aA7LCVf_ay81Htw,2.912045955657959
jAatpPkGrYK9icHzeYqmAg,2.9577152729034424
jb00cLTGmjzbMAmYN6LeRA,2.405756950378418
eAHPitw_LvyE82ZoLXuc8g,2.971311569213867
EB1Pv-zCZCaMLmQdSms_wg,3.0306456089019775
ebGAiJ5gto8phUpGU2MLwg,2.9177281856536865
ebj

 98%|█████████▊| 2442/2496 [00:04<00:00, 514.46it/s]

cFmqPhAJBgXzy3epwnx_9Q,2.409060478210449
vzp0MKUTJnMvz2JNqBWJ2g,2.919177532196045
VzZKxSSklsUMz7PuBhsZtg,2.915850877761841
V_LCGJ0yUIIqkglL_Trxng,2.9218294620513916
w-wxwyw0dus9abgR0DOD-A,2.8690807819366455
w1PSXjP-k-2F-8ZlJo1xGQ,2.4155046939849854
w2zl6iSBOm5BQoelMs9EWg,2.9244325160980225
w3C_IFq5tm6KyFHH1902ig,2.921229600906372
zMWnOn6GE-R6nnO_tGzBkw,2.4093101024627686
zn9frxkNE6uU7gTy4arVAw,2.3701331615448
ZNGKpVq8sUE2f0YfrAWuPg,2.386960506439209
ZngQZGKdUP6U-8X3QuLIpw,2.917323350906372
ZNNEAewfdtj9iCOr6dGMFg,0.0
ZPk6_i6NGLyvuQtlsJRd1w,2.4875900745391846
zpNMxm3vZaHD-oUWEhbwrw,2.407876491546631
zQfklW32VJTGI8CTL0dpsA,2.9212093353271484
kHvfW3dNAUhU6VPRGcTL6w,2.916074275970459
Kj4ks7GltRB4FQucBkzDdA,0.8373798131942749
kjDUFlKxKc--JXP1LiHGog,2.4096834659576416
KK0xHG9LXiVxhJH9SweeGw,2.9228739738464355
KKA0X2shVOI5zBNZrtbMIA,2.8877336978912354
kL6I37cK39N5rnFixX8mBQ,2.403698444366455
KLJMrwXSTN7YKYJ8q8Hg8w,2.9215097427368164
IU0Ubc1dQNj9Oa6i5C078w,2.9233462810516357
iUbIzoqkOeixkXuNRNa

100%|██████████| 2496/2496 [00:04<00:00, 513.38it/s]

DM9d0kq933WeaPmthzDZkg,1.0135343074798584
dmlahjzaamhj9jtBf2KzRg,2.353612184524536
DMwQewIU6GfNqTcclISd8Q,2.881962537765503
dNsEc2696jw0Hcqrg3tmxA,2.939276933670044
Dp9Zi2CnC-LSt2f4G1Z6gw,2.8754308223724365


In [71]:
list_to_file(distances, camera_h_file)


## rowcol_to_spherial


In [14]:
# import GPano
# import Fill_sidewalks as sw
import pandas as pd
import os
import glob

In [9]:
ground_truth = r'/media/huan/SSD/Datasets/HamptonRoads/EC-latlon-panoId.csv'
# ground_truth = r'L:\Datasets\HamptonRoads\EC-latlon-panoId.csv'
EC_df = pd.read_csv(ground_truth)
EC_df = EC_df.set_index("ID")
EC_df.loc[0]

FileNotFoundError: [Errno 2] File b'/media/huan/SSD/Datasets/HamptonRoads/EC-latlon-panoId.csv' does not exist: b'/media/huan/SSD/Datasets/HamptonRoads/EC-latlon-panoId.csv'

In [10]:
json_file = r'/media/huan/SSD/Datasets/HamptonRoads/json2500_2_DEM_camera_h.csv'
# json_file = r'L:\Datasets\HamptonRoads\json2500_2_DEM_camera_h.csv'


json_df = pd.read_csv(json_file)
json_df = json_df.set_index('panoId')
json_df

FileNotFoundError: [Errno 2] File b'/media/huan/SSD/Datasets/HamptonRoads/json2500_2_DEM_camera_h.csv' does not exist: b'/media/huan/SSD/Datasets/HamptonRoads/json2500_2_DEM_camera_h.csv'

In [11]:
for idx, row in EC_df[1:2].iterrows():
    panoId = row['panoId']
    print(panoId)
    print(json_df.loc[panoId]['camera_height_m'])

NameError: name 'EC_df' is not defined

In [31]:
import math
from math import *

def castesian_to_shperical(col, row, fov_h, height, width):  # yaw: set the heading, pitch
    """
    Convert the row, col to the  spherical coordinates 
    :param row, cols: 
    :param fov_h:
    :param height:
    :param width:
    :return:
    """
     
    col = col - width/2  # move the origin to center
    row = height/2 -row
 
    fov_v = atan((height * tan((fov_h / 2)) / width)) * 2
     
    
    r = (width/2)/tan(fov_h/2)
     
    s = sqrt(col**2 + r**2)
    
    theta = atan(row/s)
    phi = atan(col/r)
    
#     print("fov_v, r, s:", fov_v, r, s)
     
    return phi, theta

phi, theta = castesian_to_shperical(611, 445, math.pi/6, 768, 768) 

print(degrees(theta), degrees(phi))

-2.407338224132094 9.000704724021261


In [17]:
import glob
import os
import math
from PIL import Image
from tqdm import tqdm
import pandas as pd

detected_dir = r'/media/huan/HD4T/Dataset/HamptonRoads/Google_thumbnails30_detected2'
# detected_dir = r'K:\Dataset\HamptonRoads\Google_thumbnails30_detected'



dets_txts = glob.glob(os.path.join(detected_dir, "*.txt"))
print(len(dets_txts))

def is_height_floor(height_list, thres_m = 2):
    max_h = max(height_list)
    min_h = min(height_list)
    range_h = max_h - min_h
    result = [False] * len(height_list)
    print(max_h)
    if range_h > thres_m:
        print("Found doors on 2nd floor.")
        for idx, h in enumerate(height_list):
            if h - min_h > thres_m:
                result[idx] = True
            
    return result

height_list = [0.2, 0.3, 2.8, 3.0]
# is_height_floor(height_list, thres_m = 2)

def is_at_edge(x, y, w, h, edge_thres=0.01):
    
    x = float(x)
    y = float(y)
    w = float(w)
    h = float(h)
    
    top = y - h/2
    bottom = y + h/2
    left = x - w/2
    right = x + w/2
    
    if edge_thres > left:
        return "left"
    if right > (1 - edge_thres):
        return "right"  
    
    if edge_thres > top:
        return "top"
    if bottom > (1 - edge_thres):
        return "bottom"
    
  
    
    return "Not_at_edge"

# xywh:  ['0.485026', '0.432943', '0.970052', '0.613281']
is_at_edge('0.0096875', '0.99554036', '0.0625', '0.154948')

0


'left'

In [18]:
from natsort import natsorted

dets_txts = natsorted(dets_txts)

dets_txts

[]

In [10]:
target_id = 0

measurements = []
measurements.append("file,panoId,EC_id,col,bottom_row,top_row," + \
                    "FFE_gsv_m,EC_FFE_m,error_m,delta_h,distance,sink_distance,camera_h,camera_dem_m,phi")

column_names = ["image", "panoId","EC_id", "col","bottom_row","top_row",
                    "FFE_gsv_m","EC_FFE_m","error_m","delta_h","distance",
                "sink_distance","camera_h","camera_dem_m","phi", "pano_lat", "pano_lon",
               "target_lat", "target_lon", "target_elev", "image_date", 'edge']

#                 measurement_df.at[current_row, 'pano_lat'] = pano_lat
#                 measurement_df.at[current_row, 'pano_lon'] = pano_lon
# #                 measurement_df.at[current_row, 'target_lat'] = target_lat
# #                 measurement_df.at[current_row, 'target_lon'] = target_lon
#                 measurement_df.at[current_row, 'target_elev'] = FFE_gsv_m
#                 measurement_df.at[current_row, 'image_date'] = image_date 
        
measurement_df = pd.DataFrame(columns = column_names)

 

for idx, txt in enumerate(dets_txts[5000:]):
    
    basename = os.path.basename(txt)
    
    EC_id = basename.split("_")[0]
    EC_id = int(EC_id)
    
    
    
    if EC_id != 3398:
        continue
        
    print(idx, "/", len(dets_txts), EC_id)
    
    print(txt)
    
    f = open(txt, 'r')
    lines = f.readlines()
#     print(lines)
    lines = [line.replace(" \n", "") for line in lines]
#     print(lines)
    objects_fileds = [line.split(' ') for line in lines]
#     print(objects_fileds)

    fov_h = math.pi/4
    
    

    img_name = txt.replace(".txt", '.jpg')
    img = Image.open(img_name)
    
    img_w, img_h = img.size
    
    img_name = os.path.basename(txt)[:-4]
    
#     panoId = extract_panoId_frm_name(img_name)
#     panoId = extract_panoId_frm_Latlon_name(img_name)
    
    
    EC_FFE = EC_df.loc[EC_id]['FFE_ft']
    
    
    img_heading_deg = basename.replace(".txt", '').split("_")[-1] 
    
    panoId = EC_df.loc[EC_id]['panoId']

    
    image_date = json_df.loc[panoId]['image_date']

       
    camera_h = json_df.loc[panoId]['camera_height_m']
    camera_dem_m = json_df.loc[panoId]['DEM_ft'] * 0.3048
    pano_lat = json_df.loc[panoId]['lat']
    pano_lon = json_df.loc[panoId]['lng']        
    
    for o in objects_fileds:
        try:
            
#             print(idx, "/", len(dets_txts))
            
            obj_id = int(o[0])
            xywh = o[1:]
            x, y, w, h = [float(n) for n in xywh]
            
            print("xywh: ", xywh)
            min_h = -99
            
    #     print(obj_id, x, y, w, h)
            if obj_id == target_id:
                
                print(idx, "/", len(dets_txts))
                
                print("EC_id, panoId", EC_id, panoId)
                
                current_row = len(measurement_df)  

                top_row = img_h * (y - h/2)
                bottom_row = img_h * (y + h/2)
                col = img_w * x

    #             print("col:", col)

                top_agls = castesian_to_shperical(col, top_row, fov_h, img_h, img_w)
                bottom_agls = castesian_to_shperical(col, bottom_row, fov_h, img_h, img_w)
                phi = bottom_agls[0]

    #             print("top_row, bottom_row, top_agl, bottom_agl:", top_row, bottom_row, top_agls, bottom_agls)
    #             print("bottom_agl:", bottom_agls)

    #             print("degree:", degrees(bottom_agls[1]), degrees(bottom_agls[1]))

                a1 = top_agls[1]
                a2 = bottom_agls[1]
            
                print(a1, a2)
            
                print("a1, a2", degrees(a1), degrees(a2), degrees(a1 - a2))

                door_h = 2.03

                distance = door_h * cos(a1) * cos(a2) / sin(a1 - a2)  # meters

                sink_distance = distance * tan(a2)

    #             print("sink_distance:", sink_distance)

                delta_h = camera_h + sink_distance
        
                FFE_gsv_m = delta_h + camera_dem_m
            
                EC_FFE_m = EC_FFE * 0.3048
                
                error_m = abs(FFE_gsv_m - EC_FFE_m)
                
                target_heading_deg = float(img_heading_deg) + float(math.degrees(phi))
                
                edge = is_at_edge(x, y, w, h)
                
                target_lon, target_lat = \
                                      target_to_lonlat(target_heading_deg, distance, pano_lon, pano_lat)

#                 measurement = f'{img_name}.jpg,{panoId},{EC_id},{(int(col))},{int(bottom_row)},' + \
#                               f'{int(top_row)},{FFE_gsv_m:.3f},{EC_FFE_m:.3f},{error_m:.3f},' + \
#                               f'{delta_h:.3f},{distance:.3f},{sink_distance:3f},' + \
#                               f'{camera_h:.3f},{camera_dem_m:.3f},{phi}'  
    #             line = ','.join(line)
                print("target_lon, target_lat: ", target_lon, target_lat) 
                print("distance: ", distance) 
                print("error_m: ", error_m) 
                print("edge: ", edge) 
            
                measurement_df.at[current_row, 'image'] = img_name + '.jpg'
                measurement_df.at[current_row, 'panoId']   = panoId
                measurement_df.at[current_row, 'EC_id']    = EC_id
                measurement_df.at[current_row, 'col']     = int(col)
                measurement_df.at[current_row, 'bottom_row'] = int(bottom_row)
                measurement_df.at[current_row, 'top_row'] = int(top_row)
                measurement_df.at[current_row, 'FFE_gsv_m'] = float(FFE_gsv_m)
                measurement_df.at[current_row, 'EC_FFE_m'] = EC_FFE_m
                measurement_df.at[current_row, 'error_m']  = error_m
                measurement_df.at[current_row, 'delta_h']  = delta_h
                measurement_df.at[current_row, 'distance'] = distance
                measurement_df.at[current_row, 'sink_distance'] = sink_distance
                measurement_df.at[current_row, 'camera_h'] = camera_h
                measurement_df.at[current_row, 'camera_dem_m'] = camera_dem_m
                measurement_df.at[current_row, 'phi'] = phi
                measurement_df.at[current_row, 'pano_lat'] = pano_lat
                measurement_df.at[current_row, 'pano_lon'] = pano_lon
                measurement_df.at[current_row, 'target_lat'] = target_lat
                measurement_df.at[current_row, 'target_lon'] = target_lon
                measurement_df.at[current_row, 'target_elev'] = FFE_gsv_m
                measurement_df.at[current_row, 'image_date'] = image_date 
                measurement_df.at[current_row, 'edge'] = edge
  
  
#                 measurements.append(measurement)

    #             print("Higher than road surface: {:.3f}".format(delta_h))
#                 print(measurement)
        except Exception as e:
            print("Error: " , e)
            continue
            
#             print('\n')


            
measure_file = os.path.join(detected_dir, 'measurements.csv')



measurement_df.to_csv(measure_file, index=False)
# list_to_file(measurements, measure_file)

In [22]:
measurement_df["FFE_gsv_m"] = measurement_df["FFE_gsv_m"].astype(float)

lowest_door_idx = measurement_df[measurement_df['edge'] == 'Not_at_edge'].groupby("EC_id")["FFE_gsv_m"].idxmin()
lowest_door_idx

EC_id
4       1602
5       1611
14        32
22       460
29      1023
        ... 
3765    1554
3766    1558
3771    1562
3779    1566
3780    1571
Name: FFE_gsv_m, Length: 722, dtype: int64

In [23]:
measurement_df["is_lowest_door"] = 'N'
measurement_df.loc[lowest_door_idx, ["is_lowest_door"]] = "Y"    

In [24]:
measurement_df  

,image,panoId,EC_id,col,bottom_row,top_row,FFE_gsv_m,EC_FFE_m,error_m,delta_h,...,camera_dem,phi,pano_lat,pano_lon,target_lat,target_lon,target_ele,image_date,edge,is_lowest_door
0,1000_37.213599_-76.424768_0_257.08.jpg,1nObxvt2Wnf4FUtMZ8Oaow,1000,542,499,388,2.716365,2.948940,0.232575,0.827422,...,1.888943,0.110496,37.213599,-76.424768,37.213572,-76.425060,2.716365,2015-09,Not_at_edge,Y
1,1000_37.213599_-76.424768_0_272.08.jpg,1nObxvt2Wnf4FUtMZ8Oaow,1000,166,499,388,2.716365,2.948940,0.232575,0.827422,...,1.888943,-0.150619,37.213599,-76.424768,37.213572,-76.425062,2.716365,2015-09,Not_at_edge,N
2,1001_37.213599_-76.424768_0_77.08.jpg,1nObxvt2Wnf4FUtMZ8Oaow,1001,495,491,355,3.206812,2.933700,0.273112,1.317869,...,1.888943,0.077993,37.213599,-76.424768,37.213627,-76.424529,3.206812,2015-09,Not_at_edge,Y
3,1001_37.213599_-76.424768_0_92.08.jpg,1nObxvt2Wnf4FUtMZ8Oaow,1001,118,493,354,3.212399,2.933700,0.278699,1.323455,...,1.888943,-0.183185,37.213599,-76.424768,37.213627,-76.424531,3.212399,2015-09,Not_at_edge,N
4,1005_37.212088_-76.428801_0_187.68.jpg,CqiF_Zne8IwgoUqRjuGrFg,1005,590,457,312,3.434081,3.055620,0.378461,1.890748,...,1.543333,0.143449,37.212088,-76.428801,37.211914,-76.428863,3.434081,2016-05,Not_at_edge,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1718,995_37.214283_-76.423492_0_76.21.jpg,hCLb2_vr1FdvQMAkKREH8Q,995,432,517,388,2.510659,2.932786,0.422126,0.735635,...,1.775024,0.033830,37.214283,-76.423492,37.214325,-76.423243,2.510659,2016-05,Not_at_edge,N
1719,995_37.214283_-76.423492_0_91.21.jpg,hCLb2_vr1FdvQMAkKREH8Q,995,51,521,390,2.480630,2.932786,0.452156,0.705606,...,1.775024,-0.227649,37.214283,-76.423492,37.214325,-76.423241,2.480630,2016-05,Not_at_edge,Y
1720,996_37.213882_-76.423337_0_103.25.jpg,ET-hI3D_E2FJePRm-f-Nkw,996,21,474,344,3.359987,3.359506,0.000481,1.502834,...,1.857152,-0.247422,37.213882,-76.423337,37.213885,-76.423075,3.359987,2016-05,left,N
1721,996_37.213882_-76.423337_0_73.25.jpg,ET-hI3D_E2FJePRm-f-Nkw,996,607,479,343,3.343325,3.359506,0.016180,1.486173,...,1.857152,0.154368,37.213882,-76.423337,37.213909,-76.423095,3.343325,2016-05,Not_at_edge,Y


In [140]:
measurement_df.loc[lowest_door_idx]["is_lowest_door"]  

2041    Y
2097    Y
2185    Y
165     Y
234     Y
       ..
1970    Y
1978    Y
1982    Y
1983    Y
1989    Y
Name: is_lowest_door, Length: 1062, dtype: object

In [25]:
measure_file = r'K:\Dataset\HamptonRoads\Google_thumbnails30_detected2\spatial_joined_cleaned.csv'
measurement_df.to_csv(measure_file, index=False)

In [142]:
measurement_df = pd.read_csv(r'K:\Dataset\HamptonRoads\Google_thumbnails30_detected\measurements4JoinEC.csv')

In [22]:
def is_height_floor(height_list, thres_m = 2):
    max_h = max(height_list)
    min_h = min(height_list)
    range_h = max_h - min_h
    result = [False] * len(height_list)
    print(max_h)
    if range_h > thres_m:
        print("Found doors on 2nd floor.")
        for idx, h in enumerate(height_list):
            if h - min_h > thres_m:
                result[idx] = True
            
    return result

height_list = [0.2, 0.3, 2.8, 3.0]

is_height_floor(height_list, thres_m = 2)

3.0
Found doors on 2nd floor.


[False, False, True, True]

In [20]:
measurement_df = pd.read_csv(r'K:\Dataset\HamptonRoads\Google_thumbnails30_detected2\spatial_joined_cleaned.csv')
measurement_df

,image,panoId,EC_id,col,bottom_row,top_row,FFE_gsv_m,EC_FFE_m,error_m,delta_h,...,camera_h,camera_dem,phi,pano_lat,pano_lon,target_lat,target_lon,target_ele,image_date,edge
0,1000_37.213599_-76.424768_0_257.08.jpg,1nObxvt2Wnf4FUtMZ8Oaow,1000,542,499,388,2.716365,2.948940,0.232575,0.827422,...,2.929925,1.888943,0.110496,37.213599,-76.424768,37.213572,-76.425060,2.716365,2015-09,Not_at_edge
1,1000_37.213599_-76.424768_0_272.08.jpg,1nObxvt2Wnf4FUtMZ8Oaow,1000,166,499,388,2.716365,2.948940,0.232575,0.827422,...,2.929925,1.888943,-0.150619,37.213599,-76.424768,37.213572,-76.425062,2.716365,2015-09,Not_at_edge
2,1001_37.213599_-76.424768_0_77.08.jpg,1nObxvt2Wnf4FUtMZ8Oaow,1001,495,491,355,3.206812,2.933700,0.273112,1.317869,...,2.929925,1.888943,0.077993,37.213599,-76.424768,37.213627,-76.424529,3.206812,2015-09,Not_at_edge
3,1001_37.213599_-76.424768_0_92.08.jpg,1nObxvt2Wnf4FUtMZ8Oaow,1001,118,493,354,3.212399,2.933700,0.278699,1.323455,...,2.929925,1.888943,-0.183185,37.213599,-76.424768,37.213627,-76.424531,3.212399,2015-09,Not_at_edge
4,1005_37.212088_-76.428801_0_187.68.jpg,CqiF_Zne8IwgoUqRjuGrFg,1005,590,457,312,3.434081,3.055620,0.378461,1.890748,...,2.919651,1.543333,0.143449,37.212088,-76.428801,37.211914,-76.428863,3.434081,2016-05,Not_at_edge
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1718,995_37.214283_-76.423492_0_76.21.jpg,hCLb2_vr1FdvQMAkKREH8Q,995,432,517,388,2.510659,2.932786,0.422126,0.735635,...,2.828583,1.775024,0.033830,37.214283,-76.423492,37.214325,-76.423243,2.510659,2016-05,Not_at_edge
1719,995_37.214283_-76.423492_0_91.21.jpg,hCLb2_vr1FdvQMAkKREH8Q,995,51,521,390,2.480630,2.932786,0.452156,0.705606,...,2.828583,1.775024,-0.227649,37.214283,-76.423492,37.214325,-76.423241,2.480630,2016-05,Not_at_edge
1720,996_37.213882_-76.423337_0_103.25.jpg,ET-hI3D_E2FJePRm-f-Nkw,996,21,474,344,3.359987,3.359506,0.000481,1.502834,...,2.919111,1.857152,-0.247422,37.213882,-76.423337,37.213885,-76.423075,3.359987,2016-05,left
1721,996_37.213882_-76.423337_0_73.25.jpg,ET-hI3D_E2FJePRm-f-Nkw,996,607,479,343,3.343325,3.359506,0.016180,1.486173,...,2.919111,1.857152,0.154368,37.213882,-76.423337,37.213909,-76.423095,3.343325,2016-05,Not_at_edge


In [82]:
# measurement_df.to_csv(measure_file, index=False)

## localize

In [8]:

from pyproj import Transformer, Proj, transform
import math

    

def target_to_lonlat(heading_deg, distance_m, car_lon, car_lat):
    
    heading_rad = math.radians(heading_deg)
    x = distance_m * math.sin(heading_rad)
    y = distance_m * math.cos(heading_rad)
    
    csr_string = f"+proj=tmerc +lat_0={car_lat} +lon_0={car_lon} +datum=WGS84 +units=m +no_defs"
    inProj =  Proj(csr_string)
    
    outProj = Proj('epsg:4326')  # WGS84

    
    target_lat, target_lon = transform(inProj, outProj, x, y)
    
#     transformer = Transformer(inProj, outProj)
#     transformer.transform(12, 12)

    
    return target_lon, target_lat


lat = 37.213599
lon =-76.424768
distance_m = 20

target_to_lonlat(30, distance_m, lon, lat)

(-76.42465533902813, 37.21375506635809)

In [ ]:
from pyproj import Transformer, Proj, transform
import math


In [ ]:
target_to_lonlat(30, distance_m, lon, lat)

In [3]:
lat = 37.213599
lon =-76.424768

csr_string = f"+proj=tmerc +lat_0={lat} +lon_0={lon} +datum=WGS84 +units=m +no_defs"

In [15]:
local_csr = Proj(csr_string)

inProj = Proj(init='epsg:4326')  # WGS84

viewpoint = transform(inProj, local_csr, lon, lat)

viewpoint

/home/huan/anaconda3/envs/street_view/lib/python3.8/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/home/huan/anaconda3/envs/street_view/lib/python3.8/site-packages/pyproj/crs/crs.py:294: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(" ".join((projstring, projkwargs)))
<ipython-input-15-9bf82c191481>:5: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pypr

(0.0, 0.0)

In [66]:
jpg_dir = r'/media/huan/HD4T/Dataset/HamptonRoads/Google_thumbnails/*.jpg'

jpgs = glob.glob(jpg_dir)

jpgs = [os.path.basename(x) for x in jpgs]

jpg_ids = [jpg.split("_")[0] for jpg in jpgs]
list_to_file(jpg_ids, r'/media/huan/HD4T/Dataset/HamptonRoads/Google_thumbnails/jpg.txt')

In [67]:
jpg_ids

['1000',
 '1001',
 '1002',
 '1003',
 '1004',
 '1005',
 '1009',
 '1011',
 '1012',
 '1013',
 '1014',
 '1015',
 '3103',
 '3104',
 '3105',
 '3106',
 '3107',
 '3108',
 '3109',
 '310',
 '3110',
 '3111',
 '3112',
 '3113',
 '470',
 '471',
 '472',
 '473',
 '474',
 '475',
 '476',
 '477',
 '478',
 '479',
 '47',
 '480',
 '590',
 '591',
 '592',
 '595',
 '59',
 '5',
 '60',
 '611',
 '612',
 '614',
 '615',
 '616',
 '720',
 '721',
 '722',
 '723',
 '724',
 '725',
 '726',
 '727',
 '728',
 '729',
 '72',
 '730',
 '1200',
 '1205',
 '1208',
 '120',
 '1211',
 '1215',
 '1219',
 '1224',
 '1225',
 '1226',
 '1231',
 '1234',
 '1017',
 '1035',
 '1059',
 '1077',
 '1109',
 '113',
 '1166',
 '119',
 '1236',
 '1266',
 '1298',
 '1319',
 '1550',
 '1551',
 '1552',
 '1556',
 '1557',
 '155',
 '1560',
 '1564',
 '156',
 '1570',
 '1572',
 '157',
 '1760',
 '1761',
 '1762',
 '1763',
 '1764',
 '1765',
 '1766',
 '1767',
 '1768',
 '1769',
 '176',
 '1770',
 '1940',
 '1941',
 '1942',
 '1943',
 '1944',
 '1945',
 '1946',
 '1947',
 '1948

# Moving files

In [79]:
import glob
import os
import shutil

import pandas as pd

file_dir = r'K:\Dataset\HamptonRoads\Google_thumbnails30'
to_dir = r'K:\Dataset\HamptonRoads\Google_thumbnails30\missed'
files = glob.glob(os.path.join(file_dir, "*.jpg"))

cleaned_list = r'K:\Dataset\HamptonRoads\Google_thumbnails_house_only_cleaned\not_detected.csv'



In [80]:
not_clear_dir = r'K:\Dataset\HamptonRoads\Google_thumbnails_house_only_cleaned\not_detected\*.jpg'

not_clear_files = glob.glob(not_clear_dir)

In [81]:
cleaned_dict = {}
for f in not_clear_files:
    basename = os.path.basename(f)
    ec_id = basename.split("_")[0]
    cleaned_dict[ec_id] = True
    
cleaned_dict

{'1012': True,
 '101': True,
 '104': True,
 '1754': True,
 '1762': True,
 '1770': True,
 '1788': True,
 '1866': True,
 '1871': True,
 '1874': True,
 '1879': True,
 '1892': True,
 '1904': True,
 '1905': True,
 '1915': True,
 '1921': True,
 '1931': True,
 '1949': True,
 '1955': True,
 '1959': True,
 '1974': True,
 '1977': True,
 '1978': True,
 '2008': True,
 '2023': True,
 '2034': True,
 '2045': True,
 '2069': True,
 '2078': True,
 '2080': True,
 '2129': True,
 '2152': True,
 '2282': True,
 '2294': True,
 '2312': True,
 '2333': True,
 '2341': True,
 '2369': True,
 '2387': True,
 '2399': True,
 '2407': True,
 '2509': True,
 '2518': True,
 '2543': True,
 '2562': True,
 '2565': True,
 '2600': True,
 '2625': True,
 '2629': True,
 '2650': True,
 '2698': True,
 '2758': True,
 '2840': True,
 '2936': True,
 '2948': True,
 '2972': True,
 '2973': True,
 '2986': True,
 '3020': True,
 '3034': True,
 '3035': True,
 '3061': True,
 '3265': True,
 '3277': True,
 '3340': True,
 '3343': True,
 '3408': Tru

In [64]:
cleaned_df = pd.read_csv(cleaned_list)
cleaned_dict = {}

print(len(files))
print(len(cleaned_df))

785
125


In [74]:
cleaned_df = pd.read_csv(cleaned_list)
cleaned_dict = {}

for idx, row in cleaned_df.iterrows():
    ec = row['EC-ID']
    cleaned_dict[str(ec)] = True
    
print(cleaned_dict)

{'7': True, '64': True, '67': True, '69': True, '92': True, '101': True, '104': True, '211': True, '380': True, '383': True, '413': True, '420': True, '423': True, '424': True, '426': True, '432': True, '436': True, '452': True, '463': True, '467': True, '470': True, '488': True, '517': True, '849': True, '911': True, '1012': True, '1205': True, '1754': True, '1762': True, '1770': True, '1788': True, '1866': True, '1871': True, '1874': True, '1879': True, '1892': True, '1904': True, '1905': True, '1915': True, '1921': True, '1931': True, '1949': True, '1955': True, '1959': True, '1974': True, '1977': True, '1978': True, '2008': True, '2023': True, '2034': True, '2045': True, '2069': True, '2078': True, '2080': True, '2129': True, '2152': True, '2282': True, '2294': True, '2312': True, '2333': True, '2341': True, '2369': True, '2377': True, '2387': True, '2399': True, '2407': True, '2509': True, '2518': True, '2543': True, '2562': True, '2565': True, '2600': True, '2625': True, '2629': 

In [82]:
for f in files[:]:
    basename = os.path.basename(f)
    fields = basename.split("_")
    file_ec_id = fields[0]
    print(file_ec_id)
#     if not cleaned_dict.get(file_ec_id, False):
    if cleaned_dict.get(file_ec_id, False):

        new_name = os.path.join(to_dir, basename)
        shutil.copy(f, new_name)
        print(file_ec_id)
print("Done")

1000
1000
1000
1000
1000
1001
1001
1001
1001
1001
1005
1005
1005
1005
1005
1011
1011
1011
1011
1011
1012
1012
1012
1012
1012
1012
1012
1012
1012
1012
1013
1013
1013
1013
1013
1014
1014
1014
1014
1014
1015
1015
1015
1015
1015
101
101
101
101
101
101
101
101
101
101
1022
1022
1022
1022
1022
1023
1023
1023
1023
1023
104
104
104
104
104
104
104
104
104
104
108
108
108
108
108
138
138
138
138
138
144
144
144
144
144
145
145
145
145
145
146
146
146
146
146
14
14
14
14
14
1747
1747
1747
1747
1747
1748
1748
1748
1748
1748
1749
1749
1749
1749
1749
1750
1750
1750
1750
1750
1752
1752
1752
1752
1752
1753
1753
1753
1753
1753
1754
1754
1754
1754
1754
1754
1754
1754
1754
1754
1757
1757
1757
1757
1757
1758
1758
1758
1758
1758
1759
1759
1759
1759
1759
175
175
175
175
175
1761
1761
1761
1761
1761
1762
1762
1762
1762
1762
1762
1762
1762
1762
1762
1765
1765
1765
1765
1765
1766
1766
1766
1766
1766
1767
1767
1767
1767
1767
1768
1768
1768
1768
1768
1770
1770
1770
1770
1770
1770
1770
1770
1770
1770
1772
1772


2543
2543
2543
2543
2543
2546
2546
2546
2546
2546
2557
2557
2557
2557
2557
2558
2558
2558
2558
2558
2559
2559
2559
2559
2559
2561
2561
2561
2561
2561
2562
2562
2562
2562
2562
2562
2562
2562
2562
2562
2563
2563
2563
2563
2563
2564
2564
2564
2564
2564
2565
2565
2565
2565
2565
2565
2565
2565
2565
2565
2566
2566
2566
2566
2566
2567
2567
2567
2567
2567
2568
2568
2568
2568
2568
2570
2570
2570
2570
2570
2573
2573
2573
2573
2573
2577
2577
2577
2577
2577
2581
2581
2581
2581
2581
2583
2583
2583
2583
2583
2584
2584
2584
2584
2584
2585
2585
2585
2585
2585
2586
2586
2586
2586
2586
2588
2588
2588
2588
2588
2590
2590
2590
2590
2590
2595
2595
2595
2595
2595
2600
2600
2600
2600
2600
2600
2600
2600
2600
2600
2603
2603
2603
2603
2603
2604
2604
2604
2604
2604
2606
2606
2606
2606
2606
2607
2607
2607
2607
2607
2608
2608
2608
2608
2608
2609
2609
2609
2609
2609
2611
2611
2611
2611
2611
2612
2612
2612
2612
2612
2616
2616
2616
2616
2616
2622
2622
2622
2622
2622
2625
2625
2625
2625
2625
2625
2625
2625
2625
2625


3520
3520
3520
3520
3520
3520
3520
3520
3520
3521
3521
3521
3521
3521
3524
3524
3524
3524
3524
3528
3528
3528
3528
3528
353
353
353
353
353
3543
3543
3543
3543
3543
3544
3544
3544
3544
3544
3545
3545
3545
3545
3545
3559
3559
3559
3559
3559
3560
3560
3560
3560
3560
3566
3566
3566
3566
3566
3567
3567
3567
3567
3567
3571
3571
3571
3571
3571
3573
3573
3573
3573
3573
3575
3575
3575
3575
3575
3575
3575
3575
3575
3575
3576
3576
3576
3576
3576
3578
3578
3578
3578
3578
3584
3584
3584
3584
3584
3584
3584
3584
3584
3584
358
358
358
358
358
3590
3590
3590
3590
3590
3591
3591
3591
3591
3591
3593
3593
3593
3593
3593
3597
3597
3597
3597
3597
3599
3599
3599
3599
3599
3600
3600
3600
3600
3600
3604
3604
3604
3604
3604
3605
3605
3605
3605
3605
3605
3605
3605
3605
3605
3625
3625
3625
3625
3625
3633
3633
3633
3633
3633
3635
3635
3635
3635
3635
3636
3636
3636
3636
3636
3638
3638
3638
3638
3638
3651
3651
3651
3651
3651
3653
3653
3653
3653
3653
3654
3654
3654
3654
3654
3656
3656
3656
3656
3656
3656
3656
3656


In [1]:
import pandas as pd
import os
import sys
import glob

from tqdm import tqdm

import numpy as np
import geopandas as gpd
from shapely.geometry import Point
from geopandas import GeoSeries


In [2]:
targeted_file = r'K:\Dataset\Beltline\atlanta_poi1500.csv'
target_dict = {}
target_df = pd.read_csv(targeted_file)

for idx, row in target_df.iterrows():
    target_dict[row["place_id"]] = True
    

target_list = list(target_dict.keys())
# target_list

In [3]:
g = os.walk(r'K:\SafeGraph\Weekly Places Patterns (for data after 2020-06-15)\patterns\2020')

csv_files = []
for path,dir_list,file_list in g:  
    for file_name in file_list:  
        suff = file_name[-7:]
#         print(suff)
        if suff == '.csv.gz':
            csv_files.append(os.path.join(path, file_name))
print(len(csv_files))            

79


In [4]:
target_dfs = []
for file in tqdm(csv_files[:]):
    print(r"\nProcessing: {}".format(file))
    df = pd.read_csv(file)
    target_df = df[df["safegraph_place_id"].isin(target_list)]
    
    print(r"Select {} / {} rows.".format(len(target_df), len(df)))
    
    target_dfs.append(target_df)
    
target_dfs = pd.concat(target_dfs)
target_dfs.to_csv(r'K:\Dataset\Beltline\weekly_poi_after20200615.csv', index=False)

  0%|                                                                                           | 0/79 [00:00<?, ?it/s]

\nProcessing: K:\SafeGraph\Weekly Places Patterns (for data after 2020-06-15)\patterns\2020\06\24\16\patterns-part1.csv.gz


  1%|█                                                                                  | 1/79 [00:30<39:44, 30.57s/it]

Select 169 / 1042995 rows.
\nProcessing: K:\SafeGraph\Weekly Places Patterns (for data after 2020-06-15)\patterns\2020\06\24\16\patterns-part2.csv.gz


  3%|██                                                                                 | 2/79 [01:01<39:10, 30.53s/it]

...

Select 180 / 970212 rows.
\nProcessing: K:\SafeGraph\Weekly Places Patterns (for data after 2020-06-15)\patterns\2020\11\04\19\patterns-part4.csv.gz


100%|██████████████████████████████████████████████████████████████████████████████████| 79/79 [42:09<00:00, 32.02s/it]

Select 174 / 972019 rows.



D:\Anaconda3\envs\Google_street_view\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # This is added back by InteractiveShellApp.init_path()


TypeError: to_csv() got an unexpected keyword argument 'indext'

In [7]:
target_dfs.to_csv(r'K:\Dataset\Beltline\weekly_poi_after20200615.csv', index=False)

In [8]:
target_dfs2 = target_dfs[["date_range_start", 'date_range_end', 'brands', 'safegraph_place_id', \
                          'location_name', 'visitor_home_cbgs']]

In [43]:
import json
result_df = pd.DataFrame(data=None, columns=target_dfs2.columns)

for idx, row in tqdm(target_dfs2.iloc[:].iterrows()):
    visitor_home_dictstr = row['visitor_home_cbgs']
    visitor_home_dict = json.loads(visitor_home_dictstr)
#     print(visitor_home_dict)
    cbgs = visitor_home_dict.keys()
    for cbg in cbgs:
        cnt = visitor_home_dict[cbg]
#         result_df.append(row[["date_range_start", 'date_range_end', 'brands', 'safegraph_place_id', \
#                           'location_name', 'visitor_home_cbgs']])
        
        result_df.loc[len(result_df)] = row
        result_df.at[len(result_df) - 1, 'visitor_home_cbgs'] = cbg
        result_df.at[len(result_df) - 1, 'visitor_cnt'] = cnt
        
result_df['visitor_cnt'] = result_df['visitor_cnt'].astype(int)

result_df.to_csv(r'K:\Dataset\Beltline\weekly_poi_after20200615_divided.csv', index=False)

result_df        

0it [00:00, ?it/s]

{'131350504233': 5, '131210082011': 4, '132470604052': 4, '130630406201': 4, '131350504313': 4}
{}
{'131350505371': 5, '131210017001': 4, '131270001011': 4, '130459105012': 4, '131510703103': 4, '130670311183': 4, '130670312072': 4}
{'131210065004': 4}
{'131210011003': 7, '130490101001': 6, '130630405202': 5, '130890236031': 4, '130670311102': 4, '131210101081': 4, '130890232141': 4, '131350505392': 4, '130890233061': 4, '130570910054': 4, '131210116132': 4, '130970801022': 4, '130890233062': 4, '130890214122': 4, '130670303272': 4, '131210101072': 4, '131210078022': 4, '130890234182': 4, '132231203011': 4, '131210119001': 4, '132619502003': 4, '131230802001': 4, '131210094041': 4, '130670312051': 4, '130630406133': 4, '131390015011': 4, '130890233161': 4, '131210099003': 4, '130890218052': 4, '011130310001': 4, '130670302341': 4, '131350507301': 4, '131210089021': 4, '131210070011': 4, '131430104001': 4, '131210116143': 4, '130890229003': 4, '130630405152': 4, '130670313131': 4, '1307

5it [00:00, 12.14it/s]

{'132150106052': 4, '130630406082': 4, '130970803031': 4, '131210071002': 4, '131131405012': 4, '130890234251': 4, '130670302362': 4}
{'130630406161': 7, '130630405092': 4, '131210110002': 4, '131350505231': 4, '131210070013': 4, '130159608012': 4, '131210042002': 4}
{'131210032001': 111, '131210070024': 84, '131210087004': 54, '131210024001': 23, '130890217052': 20, '131210105074': 12, '131210077042': 11, '130890233031': 10, '131350505252': 9, '131350505331': 8, '131171304032': 7, '131350502182': 7, '131210035001': 7, '130890218092': 7, '130670312072': 7, '130630406121': 7, '130630405242': 7, '130890232102': 6, '130670311111': 6, '130670302331': 6, '131210112013': 6, '131510701062': 6, '130890219094': 6, '131210105143': 6, '131299705001': 6, '131510703102': 6, '130131803022': 5, '131350507262': 5, '132450102043': 5, '360470892001': 5, '131210114184': 5, '130670303311': 5, '120570115042': 5, '131510704042': 5, '420454037012': 5, '130890228001': 5, '131210002004': 5, '130970801022': 4, 

13it [00:02,  6.49it/s]

...

14207it [35:51,  4.90it/s]

{'131210007002': 6}
{}
{'131350504232': 4}
{'120990077492': 4, '131210010021': 4}
{}
{'010690403022': 4, '131210100023': 4, '130970804043': 4, '131350502123': 4, '130570910053': 4, '131350503111': 4}

14212it [35:51,  6.60it/s]



{}
{}
{}


,date_range_start,date_range_end,brands,safegraph_place_id,location_name,visitor_home_cbgs,visitor_cnt
0,2020-06-15T00:00:00-04:00,2020-06-22T00:00:00-04:00,Texaco,sg:66c59f90200042199a1b41d379495d12,Texaco,131350504233,5
1,2020-06-15T00:00:00-04:00,2020-06-22T00:00:00-04:00,Texaco,sg:66c59f90200042199a1b41d379495d12,Texaco,131210082011,4
2,2020-06-15T00:00:00-04:00,2020-06-22T00:00:00-04:00,Texaco,sg:66c59f90200042199a1b41d379495d12,Texaco,132470604052,4
3,2020-06-15T00:00:00-04:00,2020-06-22T00:00:00-04:00,Texaco,sg:66c59f90200042199a1b41d379495d12,Texaco,130630406201,4
4,2020-06-15T00:00:00-04:00,2020-06-22T00:00:00-04:00,Texaco,sg:66c59f90200042199a1b41d379495d12,Texaco,131350504313,4
...,...,...,...,...,...,...,...
85695,2020-10-26T00:00:00-04:00,2020-11-02T00:00:00-05:00,Piedmont Healthcare,sg:063b76914f904ef1981a26e5f3e5df1b,Piedmont Healthcare,131210100023,4
85696,2020-10-26T00:00:00-04:00,2020-11-02T00:00:00-05:00,Piedmont Healthcare,sg:063b76914f904ef1981a26e5f3e5df1b,Piedmont Healthcare,130970804043,4
85697,2020-10-26T00:00:00-04:00,2020-11-02T00:00:00-05:00,Piedmont Healthcare,sg:063b76914f904ef1981a26e5f3e5df1b,Piedmont Healthcare,131350502123,4
85698,2020-10-26T00:00:00-04:00,2020-11-02T00:00:00-05:00,Piedmont Healthcare,sg:063b76914f904ef1981a26e5f3e5df1b,Piedmont Healthcare,130570910053,4


In [38]:
result_df.to_csv(r'K:\Dataset\Beltline\weekly_poi_after20200615_divided.csv', index=False)

In [44]:
result_df.dtypes

date_range_start      object
date_range_end        object
brands                object
safegraph_place_id    object
location_name         object
visitor_home_cbgs     object
visitor_cnt            int32
dtype: object

In [48]:
import glob
import os
import shutil

import pandas as pd

file_dir = r'L:\Datasets\ocean_city\street_data\images'
to_dir = r'K:\Research\Lowest_floor_toGitHub\street_data\images'
files = glob.glob(os.path.join(file_dir, "*.jpg"))

 

In [52]:
label_dir = r'K:\Research\Lowest_floor_toGitHub\street_data\labels\*.txt'

In [53]:
labels = glob.glob(label_dir)

In [54]:
for l in labels:
    basename = os.path.basename(l)
    basename = basename.replace(".txt", '.jpg')
    
    jpg = os.path.join(file_dir, basename)
    new_name = os.path.join(to_dir, basename)
    print(jpg, new_name)
    shutil.copy(jpg, new_name)
    
    

L:\Datasets\ocean_city\street_data\images\13786.jpg K:\Research\Lowest_floor_toGitHub\street_data\images\13786.jpg
L:\Datasets\ocean_city\street_data\images\13788.jpg K:\Research\Lowest_floor_toGitHub\street_data\images\13788.jpg
L:\Datasets\ocean_city\street_data\images\13789.jpg K:\Research\Lowest_floor_toGitHub\street_data\images\13789.jpg
L:\Datasets\ocean_city\street_data\images\13790.jpg K:\Research\Lowest_floor_toGitHub\street_data\images\13790.jpg
L:\Datasets\ocean_city\street_data\images\13791.jpg K:\Research\Lowest_floor_toGitHub\street_data\images\13791.jpg
L:\Datasets\ocean_city\street_data\images\13792.jpg K:\Research\Lowest_floor_toGitHub\street_data\images\13792.jpg
L:\Datasets\ocean_city\street_data\images\13795.jpg K:\Research\Lowest_floor_toGitHub\street_data\images\13795.jpg
L:\Datasets\ocean_city\street_data\images\13797.jpg K:\Research\Lowest_floor_toGitHub\street_data\images\13797.jpg
L:\Datasets\ocean_city\street_data\images\13799.jpg K:\Research\Lowest_floor_toG

L:\Datasets\ocean_city\street_data\images\29104.jpg K:\Research\Lowest_floor_toGitHub\street_data\images\29104.jpg
L:\Datasets\ocean_city\street_data\images\29105.jpg K:\Research\Lowest_floor_toGitHub\street_data\images\29105.jpg
L:\Datasets\ocean_city\street_data\images\29106.jpg K:\Research\Lowest_floor_toGitHub\street_data\images\29106.jpg
L:\Datasets\ocean_city\street_data\images\29108.jpg K:\Research\Lowest_floor_toGitHub\street_data\images\29108.jpg
L:\Datasets\ocean_city\street_data\images\29109.jpg K:\Research\Lowest_floor_toGitHub\street_data\images\29109.jpg
L:\Datasets\ocean_city\street_data\images\31072.jpg K:\Research\Lowest_floor_toGitHub\street_data\images\31072.jpg
L:\Datasets\ocean_city\street_data\images\31074.jpg K:\Research\Lowest_floor_toGitHub\street_data\images\31074.jpg
L:\Datasets\ocean_city\street_data\images\31075.jpg K:\Research\Lowest_floor_toGitHub\street_data\images\31075.jpg
L:\Datasets\ocean_city\street_data\images\31076.jpg K:\Research\Lowest_floor_toG

L:\Datasets\ocean_city\street_data\images\31161.jpg K:\Research\Lowest_floor_toGitHub\street_data\images\31161.jpg
L:\Datasets\ocean_city\street_data\images\31162.jpg K:\Research\Lowest_floor_toGitHub\street_data\images\31162.jpg
L:\Datasets\ocean_city\street_data\images\31163.jpg K:\Research\Lowest_floor_toGitHub\street_data\images\31163.jpg
L:\Datasets\ocean_city\street_data\images\31164.jpg K:\Research\Lowest_floor_toGitHub\street_data\images\31164.jpg
L:\Datasets\ocean_city\street_data\images\31165.jpg K:\Research\Lowest_floor_toGitHub\street_data\images\31165.jpg
L:\Datasets\ocean_city\street_data\images\31166.jpg K:\Research\Lowest_floor_toGitHub\street_data\images\31166.jpg
L:\Datasets\ocean_city\street_data\images\31167.jpg K:\Research\Lowest_floor_toGitHub\street_data\images\31167.jpg
L:\Datasets\ocean_city\street_data\images\31168.jpg K:\Research\Lowest_floor_toGitHub\street_data\images\31168.jpg
L:\Datasets\ocean_city\street_data\images\31169.jpg K:\Research\Lowest_floor_toG

L:\Datasets\ocean_city\street_data\images\31244.jpg K:\Research\Lowest_floor_toGitHub\street_data\images\31244.jpg
L:\Datasets\ocean_city\street_data\images\31245.jpg K:\Research\Lowest_floor_toGitHub\street_data\images\31245.jpg
L:\Datasets\ocean_city\street_data\images\31246.jpg K:\Research\Lowest_floor_toGitHub\street_data\images\31246.jpg
L:\Datasets\ocean_city\street_data\images\31247.jpg K:\Research\Lowest_floor_toGitHub\street_data\images\31247.jpg
L:\Datasets\ocean_city\street_data\images\31252.jpg K:\Research\Lowest_floor_toGitHub\street_data\images\31252.jpg
L:\Datasets\ocean_city\street_data\images\31253.jpg K:\Research\Lowest_floor_toGitHub\street_data\images\31253.jpg
L:\Datasets\ocean_city\street_data\images\31254.jpg K:\Research\Lowest_floor_toGitHub\street_data\images\31254.jpg
L:\Datasets\ocean_city\street_data\images\31255.jpg K:\Research\Lowest_floor_toGitHub\street_data\images\31255.jpg
L:\Datasets\ocean_city\street_data\images\31256.jpg K:\Research\Lowest_floor_toG

L:\Datasets\ocean_city\street_data\images\31341.jpg K:\Research\Lowest_floor_toGitHub\street_data\images\31341.jpg
L:\Datasets\ocean_city\street_data\images\31342.jpg K:\Research\Lowest_floor_toGitHub\street_data\images\31342.jpg
L:\Datasets\ocean_city\street_data\images\31344.jpg K:\Research\Lowest_floor_toGitHub\street_data\images\31344.jpg
L:\Datasets\ocean_city\street_data\images\31345.jpg K:\Research\Lowest_floor_toGitHub\street_data\images\31345.jpg
L:\Datasets\ocean_city\street_data\images\31346.jpg K:\Research\Lowest_floor_toGitHub\street_data\images\31346.jpg
L:\Datasets\ocean_city\street_data\images\31347.jpg K:\Research\Lowest_floor_toGitHub\street_data\images\31347.jpg
L:\Datasets\ocean_city\street_data\images\31348.jpg K:\Research\Lowest_floor_toGitHub\street_data\images\31348.jpg
L:\Datasets\ocean_city\street_data\images\31349.jpg K:\Research\Lowest_floor_toGitHub\street_data\images\31349.jpg
L:\Datasets\ocean_city\street_data\images\31350.jpg K:\Research\Lowest_floor_toG

L:\Datasets\ocean_city\street_data\images\31442.jpg K:\Research\Lowest_floor_toGitHub\street_data\images\31442.jpg
L:\Datasets\ocean_city\street_data\images\31447.jpg K:\Research\Lowest_floor_toGitHub\street_data\images\31447.jpg
L:\Datasets\ocean_city\street_data\images\31448.jpg K:\Research\Lowest_floor_toGitHub\street_data\images\31448.jpg
L:\Datasets\ocean_city\street_data\images\31449.jpg K:\Research\Lowest_floor_toGitHub\street_data\images\31449.jpg
L:\Datasets\ocean_city\street_data\images\31450.jpg K:\Research\Lowest_floor_toGitHub\street_data\images\31450.jpg
L:\Datasets\ocean_city\street_data\images\31452.jpg K:\Research\Lowest_floor_toGitHub\street_data\images\31452.jpg
L:\Datasets\ocean_city\street_data\images\31454.jpg K:\Research\Lowest_floor_toGitHub\street_data\images\31454.jpg
L:\Datasets\ocean_city\street_data\images\31455.jpg K:\Research\Lowest_floor_toGitHub\street_data\images\31455.jpg
L:\Datasets\ocean_city\street_data\images\31456.jpg K:\Research\Lowest_floor_toG

L:\Datasets\ocean_city\street_data\images\31559.jpg K:\Research\Lowest_floor_toGitHub\street_data\images\31559.jpg
L:\Datasets\ocean_city\street_data\images\31561.jpg K:\Research\Lowest_floor_toGitHub\street_data\images\31561.jpg
L:\Datasets\ocean_city\street_data\images\31562.jpg K:\Research\Lowest_floor_toGitHub\street_data\images\31562.jpg
L:\Datasets\ocean_city\street_data\images\31564.jpg K:\Research\Lowest_floor_toGitHub\street_data\images\31564.jpg
L:\Datasets\ocean_city\street_data\images\31565.jpg K:\Research\Lowest_floor_toGitHub\street_data\images\31565.jpg
L:\Datasets\ocean_city\street_data\images\31566.jpg K:\Research\Lowest_floor_toGitHub\street_data\images\31566.jpg
L:\Datasets\ocean_city\street_data\images\31570.jpg K:\Research\Lowest_floor_toGitHub\street_data\images\31570.jpg
L:\Datasets\ocean_city\street_data\images\31572.jpg K:\Research\Lowest_floor_toGitHub\street_data\images\31572.jpg
L:\Datasets\ocean_city\street_data\images\31573.jpg K:\Research\Lowest_floor_toG

L:\Datasets\ocean_city\street_data\images\31658.jpg K:\Research\Lowest_floor_toGitHub\street_data\images\31658.jpg
L:\Datasets\ocean_city\street_data\images\31661.jpg K:\Research\Lowest_floor_toGitHub\street_data\images\31661.jpg
L:\Datasets\ocean_city\street_data\images\31663.jpg K:\Research\Lowest_floor_toGitHub\street_data\images\31663.jpg
L:\Datasets\ocean_city\street_data\images\31665.jpg K:\Research\Lowest_floor_toGitHub\street_data\images\31665.jpg
L:\Datasets\ocean_city\street_data\images\31668.jpg K:\Research\Lowest_floor_toGitHub\street_data\images\31668.jpg
L:\Datasets\ocean_city\street_data\images\31669.jpg K:\Research\Lowest_floor_toGitHub\street_data\images\31669.jpg
L:\Datasets\ocean_city\street_data\images\31672.jpg K:\Research\Lowest_floor_toGitHub\street_data\images\31672.jpg
L:\Datasets\ocean_city\street_data\images\31673.jpg K:\Research\Lowest_floor_toGitHub\street_data\images\31673.jpg
L:\Datasets\ocean_city\street_data\images\31674.jpg K:\Research\Lowest_floor_toG

L:\Datasets\ocean_city\street_data\images\31776.jpg K:\Research\Lowest_floor_toGitHub\street_data\images\31776.jpg
L:\Datasets\ocean_city\street_data\images\31778.jpg K:\Research\Lowest_floor_toGitHub\street_data\images\31778.jpg
L:\Datasets\ocean_city\street_data\images\31779.jpg K:\Research\Lowest_floor_toGitHub\street_data\images\31779.jpg
L:\Datasets\ocean_city\street_data\images\31780.jpg K:\Research\Lowest_floor_toGitHub\street_data\images\31780.jpg
L:\Datasets\ocean_city\street_data\images\31781.jpg K:\Research\Lowest_floor_toGitHub\street_data\images\31781.jpg
L:\Datasets\ocean_city\street_data\images\31783.jpg K:\Research\Lowest_floor_toGitHub\street_data\images\31783.jpg
L:\Datasets\ocean_city\street_data\images\31784.jpg K:\Research\Lowest_floor_toGitHub\street_data\images\31784.jpg
L:\Datasets\ocean_city\street_data\images\31786.jpg K:\Research\Lowest_floor_toGitHub\street_data\images\31786.jpg
L:\Datasets\ocean_city\street_data\images\31787.jpg K:\Research\Lowest_floor_toG

# jsons_to_list

In [84]:
import math

col = 1491
  
row = 2101
# row = 1856
fov_h = math.pi/2
height = 4096
width = 4096
castesian_to_shperical(col, row, fov_h, height, width)

(-0.26554954089837773, -0.024966621041549453)

In [90]:
def jsons_to_list(jsons_list, saved_name):
    list_txt = open(saved_name, 'w', encoding="utf-8")
    list_txt.writelines('image_width,image_height,tile_width,tile_height,image_date,imagery_type,\
projection_type,pano_yaw_deg,tilt_yaw_deg,tilt_pitch_deg,panoId,zoomLevels,lat,lng,original_lat,original_lng,elevation_wgs84_m,\
description,streetRange,region,country,elevation_egm96_m,\
links_cnt,link_forward_yawDeg,link_f_panoId,link_f_road_argb,link_f_description,\
link_backward_yawDeg,link_b_panoId,link_b_road_argb,link_b_description,link_others')
    for idx, file in tqdm(enumerate(jsons_list)):
        try:
            f = open(file, 'r')
            json_data = json.load(f)

            # default values
            image_width = '0'
            image_height = '0'
            tile_width = '0'
            tile_height =  '0'
            image_date =  '0000-00'
            imagery_type =  '1'        

            projection_type = '-1'
            pano_yaw_deg =  '-999'
            tilt_yaw_deg =  '-999'
            tilt_pitch_deg =  '-999'

            panoId = '0'
            zoomLevels = '-1'
            lat = '-999'
            lng = '-999'
            original_lat = '-999'
            original_lng = '-999'
            elevation_wgs84_m = '-9999'
            description = '-'
            streetRange = '-'
            region = '-'
            country = '-'
            elevation_egm96_m = '-9999'


            Data = json_data.get('Data')
            if Data:     
                image_width =           Data.get('image_width', image_width)
                image_height =          Data.get('image_height', image_height)
                tile_width =            Data.get('tile_width', tile_width)
                tile_height =           Data.get('tile_height', tile_height)
                image_date =            Data.get('image_date', image_date)
                imagery_type =          Data.get('imagery_type', imagery_type)        

            Projection =  json_data.get('Projection')
            if Projection:
                projection_type = Projection.get('projection_type', projection_type)
                pano_yaw_deg =    Projection.get('pano_yaw_deg', pano_yaw_deg)
                tilt_yaw_deg =    Projection.get('tilt_yaw_deg', tilt_yaw_deg)
                tilt_pitch_deg =  Projection.get('tilt_pitch_deg', tilt_pitch_deg)

            Location = json_data.get('Location')
            if Location:            
                panoId =            Location.get('panoId', panoId)
                zoomLevels =        Location.get('zoomLevels', zoomLevels)
                lat =               Location.get('lat', lat)
                lng =               Location.get('lng', lng)
                original_lat =      Location.get('original_lat', original_lat)
                original_lng =      Location.get('original_lng', original_lng)
                elevation_wgs84_m = Location.get('elevation_wgs84_m', elevation_wgs84_m)
                description =       Location.get('description', description).replace(",", ';')
                streetRange =       Location.get('streetRange', streetRange)   
                region =            Location.get('region', region).replace(",", ';')
                country =           Location.get('country', country).replace(",", ';')
                elevation_egm96_m = Location.get('elevation_egm96_m', elevation_egm96_m)  

            Links = json_data.get('Links')
            
            contents = [image_width,image_height,tile_width,tile_height,image_date,imagery_type,\
projection_type,pano_yaw_deg,tilt_yaw_deg,tilt_pitch_deg,\
panoId,zoomLevels,lat,lng,original_lat,original_lng,elevation_wgs84_m,description,streetRange,region,country,elevation_egm96_m]
            
            contents = [str(x) for x in contents]
#             f.writelines(','.encode('utf-8', 'ignore'))
            list_txt.writelines('\n' + ','.join(contents))
    
            if Links:
                Cnt_links = len(Links)
                
                list_txt.writelines( ',' + str(Cnt_links))
                        
                # find the forward_link  
                yaw_in_links = [float(link['yawDeg']) for link in Links]
                pano_yaw_deg = float(pano_yaw_deg)
                diff = [abs(yawDeg - pano_yaw_deg) for yawDeg in yaw_in_links]                
                idx_min = diff.index(min(diff))
                forward_link = Links[idx_min]
                
                # find the backward_link                       
                idx_max = diff.index(max(diff))
                backward_link = Links[idx_max]
                
#                 print(idx_max, idx_min)
                        
                for link in [forward_link, backward_link]:
                    yawDeg =  link.get('yawDeg', '-999')
                    panoId =  link.get('panoId', '0')
                    road_argb =  link.get('road_argb', '0x80fdf872')
                    description =  link.get('description', '-').replace(",", ';')
                    if description == '':
                        description = '-'
                    contents = [yawDeg, panoId, road_argb, description]
                    contents = ',' + ','.join(contents)
                    list_txt.writelines(contents)
                               
                
                if len(Links) > 2:
                    if idx_max < idx_min:
                        idx_max, idx_min = idx_min ,idx_max
                        
                    Links.pop(idx_max)
                    Links.pop(idx_min)
                    
                    list_txt.writelines(',')
                    content_list = []
                    for link in Links:
                        yawDeg =  link.get('yawDeg', '-999')
                        panoId =  link.get('panoId', '0')
                        road_argb =  link.get('road_argb', '0x80fdf872')
                        description =  link.get('description', '-').replace(",", ';')
                        if description == '':
                            description = '-'
                        contents = [yawDeg, panoId, road_argb, description]
                        content_list.append('|'.join(contents))
                    
                    list_txt.writelines('|'.join(content_list))                     

        except Exception as e:
            print("Error in reading json:", file, '-- ', e)
            continue
    list_txt.close()

In [94]:
csv_file = r'/media/huan/SSD/Datasets/HamptonRoads/depthmaps/json2500_2.csv'
json_dir = r'/media/huan/SSD/Datasets/HamptonRoads/depthmaps'
jsons = glob.glob(json_dir + "/*.json")

jsons_to_list(jsons, csv_file)

2498it [00:02, 1161.23it/s]


In [3]:
# rename:
import os
import glob


files_dir = r'/media/huan/HD4T/Dataset/InterCross/IntersectionImages/*.JPG'

files = glob.glob(files_dir )

for f in files:
    os.rename(f, f.replace(".JPG", ".jpg"))